<h1 style = "font-size: 30px; text-align: center;">AI Genetic Hands On</h1>
<h2 style = "font-size: 25px; text-align: center;">Hospital Job Scheduling</h2>
<h2 style = "font-size: 25px; text-align: center; color: #666">Name: Hamid Mohtadi Jafari</h2>
<h2 style = "font-size: 25px; text-align: center; color: #666">Student Id: 810195485</h2>
<h4 style="text-align: center">Spring 1400</h4>

<h2 style = "font-size: 25px;">Notes</h2>

<h3>Tests: </h3>

<p style="text-indent :2em;"><b>Test1: </b><mark>Average time < 15s</mark></p>
<p style="text-indent :2em;"><b>Test2: </b><mark>Average time < 60s</mark></p>
    

<h3>Outputs: </h3>
<p style="text-indent :2em;">Respectively <mark>output1.txt</mark> and <mark>output2.txt</mark></p>

<br>

In [188]:
import random
import itertools

<h2 style = "font-size: 25px;">Test Files</h2>

In [189]:
testFile1 = "test1.txt"
testFile2 = "test2.txt"

<h2 style = "font-size: 25px;">Reading Test File Content</h2>

In [190]:
def readInput(testFile) :
    file = open(testFile, 'r+')
    fileList = file.readlines()
    fileList = [s.replace('\n', '') for s in fileList]
    
    [days, doctors] = [int(i) for i in fileList[0].split()]
    maxCapacity = int(fileList[1])
    
    allShifts = []
    for i in range(2, days + 2):
        dayRequirements = fileList[i].split()
        morningReqs = [int(i) for i in dayRequirements[0].split(",")]
        eveningReqs = [int(i) for i in dayRequirements[1].split(",")]
        nightReqs = [int(i) for i in dayRequirements[2].split(",")]
        
        allShifts.append((morningReqs, eveningReqs, nightReqs))

    file.close()
    return [days, list(range(doctors)), maxCapacity, allShifts]

# genes and chromosomes

In this problem the gene will be a day Schedule and  chromosome will be the whole Schedule

# First Generation

for making a first generation we use random data made with random.sample for each gene in the function make_gene but there is 2 constraints for data:
1) doctor for each shift should be unique(1 doctot can't count as 2) <br>
2) number of docs in 1 shift is between min and max of that shift(because it is easy to perform this limit )<br>
the Population is 500

# Fitness

To calculate the fitness of each chromosomes we need to check that this 3 conditions won't happen:
1) compare night shifts with morning and evening shifts of tommorow<br>
2) check for 3 continuous for same doctor<br>
3) check number of shifts for all doctors to check if it is surpassed the limit<br>
for each violation of this 3 conditions there will be negetive points to fitness and we reach the perfect chromosome when fitness score will be 0<br>
function calculateFitness used to perform this 3 conditions

# Crossover

to make new chromosomes from 2 original chromosomes we use crossover and rules are:<br>
there is 65 percent chance that crossover happens <br>
for each genes in chromosomes there is a 50 percent chance that it would change to the other chromosome<br>
you can check this in 2 function of generateNewPopulation and crossOver

# Mutation

if we only use crossover to make new chromosomes we can't add new things to population and we may reach a point that crossover can't help to rach new answers<br>
to Prevent that we use mutation:<br>
there is 10 percent chance that a random gene in chromosome would change to a completly new gene<br>
you can check this in 2 function of generateNewPopulation and mutate

# Genetic Algorithm

after Generating first generation we sort chromosomes by fitness then we check if we have a fitness equal to 0 and if we don't then on each step we do a crossover and mutation on chromosomes then we check it again until we reach max fitness<br>
the max number for each step is 3000<br>
before crossover and mutation we store 16 percent of  Population with best fitness score for new generations

<h2 style = "font-size: 25px;">Job Scheduler</h2>

In [197]:
class JobScheduler:
    def __init__(self, fileInfo):
        self.days = fileInfo[0]
        self.doctors = len(fileInfo[1])
        self.doctorsIds = fileInfo[1]
        self.maxCapacity = fileInfo[2]
        self.allShifts = fileInfo[3]
        self.popSize = 500
        self.chromosomes = self.generateInitialPopulation()
        self.reset_times = 3000
        
        # self.crossOverPoints = (relative to number of days)
        self.elitismPercentage = 0.16 #(move x% best of parents directly to the new population)
        self.pc = 0.65 
        self.pm = 0.1  
        
        
    
    def make_gene(self,day_limits):
        gene = []
        for limit in day_limits:
            doc_num = random.randint(limit[0],limit[1])
            day_souloution = random.sample(self.doctorsIds, doc_num)
            gene.append(day_souloution)
        return gene
    
    def make_gene_for_mutation(self,day_limits):
        gene = []
        for limit in day_limits:
            day_souloution = random.sample(self.doctorsIds, limit[0])
            gene.append(day_souloution)
        return gene
            
            
        
        
        
    def generateInitialPopulation(self):
        chromosomes = []
        
        for i in range(self.popSize):
            chromosome = []
            for day_shift in self.allShifts:
                chromosome.append(self.make_gene(day_shift))
            chromosome =[chromosome,0]
            fitness_score = self.calculateFitness(chromosome)
            chromosomes.append([chromosome[0],fitness_score])
        return chromosomes
        
    
    def crossOver(self, chromosome1, chromosome2):
        if random.uniform(0, 1) < self.pc:
            for i in range(len(chromosome1[0])):
                if random.uniform(0, 1) < 0.5:
                    temp = chromosome1[0][i]
                    chromosome1[0][i] = chromosome2[0][i]
                    chromosome2[0][i] = temp
        else:
            return [chromosome1, chromosome2]
        return [chromosome1, chromosome2]
        
                
    def mutate(self, chromosome):
        if random.uniform(0, 1) < self.pm:
            first = random.sample(range(len(chromosome[0])), 1)
            first = first[0]
            chromosome[0][first] = self.make_gene_for_mutation(self.allShifts[first])
        else:
            return chromosome
        return chromosome
        
       
    def check_same_doctor_on_two_continuous_days(self, schedule):
        fitness_score = 0
        for i,day_shift in enumerate(schedule):
            if i == (len(schedule)-1):
                continue
            for night_shift in day_shift[2]:
                if night_shift in schedule[i+1][0]:
                    fitness_score -= 1
                if night_shift in schedule[i+1][1]:
                    fitness_score -= 1
        return fitness_score
    
    def check_same_doctor_on_three_continuous_night_shifts(self,schedule):
        fitness_score = 0
        for i,day_shift in enumerate(schedule):
            if (i == (len(schedule)-1)) or (i == (len(schedule)-2)):
                continue
            for night_shift in day_shift[2]:
                if (night_shift in schedule[i+1][2]) and (night_shift in schedule[i+2][2]):
                    fitness_score -= 1
        return fitness_score
    
    def check_shift_limits_for_each_doctor(self,schedule):
        fitness_score = 0
        doctor_shifts = {}
        for doctor_id in self.doctorsIds:
            doctor_shifts[doctor_id] = 0
        schedule = list(itertools.chain.from_iterable(schedule))
        schedule = list(itertools.chain.from_iterable(schedule))
        for doctor_id in schedule:
            doctor_shifts[doctor_id] += 1
        for doctor_id in self.doctorsIds:
            if doctor_shifts[doctor_id] > self.maxCapacity:
                fitness_score += (self.maxCapacity - doctor_shifts[doctor_id])
        return fitness_score
            
        
    
    def calculateFitness(self, chromosome):
        fitness_score = 0
        fitness_score += self.check_same_doctor_on_two_continuous_days(chromosome[0])
        fitness_score += self.check_same_doctor_on_three_continuous_night_shifts(chromosome[0])
        fitness_score += self.check_shift_limits_for_each_doctor(chromosome[0])
        
        return fitness_score
    
    
    def generateNewPopulation(self):
        new_generation = []
        chosen_ones = int(self.elitismPercentage * self.popSize)
        for i in range(chosen_ones) : 
            new_generation.append(self.chromosomes[i])
        for i in range(int((self.popSize-chosen_ones)/2)) :
            while True:
                parent1, parent2 = random.choices(self.chromosomes[:int(self.popSize * 0.6)], k=2)
                if parent1 != parent2:
                    break
            child1, child2 = self.crossOver(parent1, parent2)
            child1 = self.mutate(child1)
            child2 = self.mutate(child2)
            child1[1] = self.calculateFitness(child1)
            child2[1] = self.calculateFitness(child2)
            new_generation.append(child1)
            new_generation.append(child2)
        new_generation.sort(key=lambda x: x[1], reverse=True)
        return new_generation
    
    
    def schedule(self): 
        self.chromosomes.sort(key=lambda x: x[1], reverse=True)
        index = 0
        while (self.chromosomes[0][1] != 0):
            if index == self.reset_times:
                break
            self.chromosomes = self.generateNewPopulation()
            index += 1
        print(index)
#         print(self.chromosomes[0])
        return self.chromosomes[0]

# Write Output

In the end we should write down the output in a file code below first open a file given as input then write on it and then close it

In [192]:
def output(schedule,name):
    f= open(name,"w+")
    for day_shift in schedule:
        line = ''
        for shift in day_shift:
            for i,doctor_id in enumerate(shift):
                line += str(doctor_id)
                if i != (len(shift)-1):
                    line += ','
            line += ' '
        line += '\n'    
        f.write(line)
    f.close()
    f.close()

<h2 style = "font-size: 25px;">Execution</h2>

In [195]:
import time


fileInfo1 = readInput(testFile1)

start = time.time()

scheduler = JobScheduler(fileInfo1)
schedule = scheduler.schedule()

end = time.time()

print("test 1: ", '%.2f'%(end - start), 'sec')
print(schedule)
output(schedule[0],"out1.txt")

19
test 1:  0.28 sec
[[[[6, 0], [2], [0, 6]], [[1], [2, 1], [6]], [[1], [4], [4]], [[1], [6], [5, 1, 6]], [[0], [], [3]], [[5], [], [4]], [[3, 1], [5], [0]], [[], [5], [2, 0]], [[], [], []], [[0], [2], [4, 1]]], 0]


In [196]:
fileInfo2 = readInput(testFile2)

start = time.time()

scheduler = JobScheduler(fileInfo2)
schedule = scheduler.schedule()

end = time.time()

print("Test 2: ", '%.2f'%(end - start), 'sec')
print(schedule)
output(schedule[0],"out2.txt")

360
Test 2:  5.68 sec
[[[[4, 3], [6], [3]], [[2, 5], [7], [6]], [[2], [8], [2, 4]], [[], [5], [6, 4]], [[2], [], [3]], [[1, 2], [5], [0, 5]], [[], [4], [7, 8]], [[2], [0, 2, 4], [1, 2]], [[8], [6, 4, 3, 5], [3, 2]], [[], [], []], [[], [6], []], [[4, 8], [7], []], [[2], [7, 6], [8, 0]], [[3], [5, 2], [2, 8, 4]], [[], [5, 7], [3]]], 0]
